In [1]:
# Importing important libraries

import os
import json

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.utils.data_utils import get_file
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, load_img

import h5py
import numpy as np
import pickle as pk

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading models for all checks
first_check = VGG16(weights='imagenet')
second_check = pk.load(open('C:/Users/admin/Desktop/Computer Vision + Django/car_damage_check/classifier.pickle', 'rb')) # damaged or not
third_check = pk.load(open('C:/Users/admin/Desktop/Computer Vision + Django/car_damage_check/FRS/classifier.pickle', 'rb')) # location of damage
forth_check = pk.load(open('C:/Users/admin/Desktop/Computer Vision + Django/car_damage_check/Severity/classifier.pickle', 'rb')) # Severity of damage

In [4]:
# Loading valid categories for identifying cars using VGG16
with open('cat_counter.pk', 'rb') as f:
    cat_counter = pk.load(f)

In [5]:
# Shortlisting top 27 categories that VGG16 stores for cars (Can be altered)
cat_list = [k for k, v in cat_counter.most_common()[:27]]

First Check - Car or not

In [6]:
CLASS_INDEX = None

In [7]:
CLASS_INDEX_PATH = 'C:/Users/admin/Desktop/Computer Vision + Django/imagenet_class_index.json'

In [8]:
def get_predictions(preds, top=5):
    global CLASS_INDEX
    
    # Load the json file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    # get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [9]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    return x

In [10]:
def car_categories_check(img_224, model):
    print("Validating that this is a picture of car or not")
    out = model.predict(img_224)
    top = get_predictions(out, top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            print("Car Check Passed")
            print("\n")
            return True
    return False

Second Check - Car damaged or not

In [11]:
def car_damage_check(classifier):
    print("Validating that damage exists....")
    base_model = first_check
    train_labels = ['Damaged', 'Not Damaged']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size = image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis = 0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    
    if train_labels[preds[0]] == 'Damaged':
        print("Validation complete - proceeding to location and severity determination")
        print("\n")
        return True
    else:
        return False

Third Check - Location of Damage

In [12]:
def location_assessment(classifier):
    print("Validating the damage area - Front, Rear or Side")
    base_model = first_check
    train_labels = ['Front', 'Rear', 'Side']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size = image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x=preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print("Your car is damaged from " + train_labels[preds[0]])
    print("Location Check Complete")
    print("\n")

Fourth Check - Severity of damage

In [13]:
def severity_assessment(classifier):
    print("Validating the severity....")
    base_model = first_check
    train_labels = ['Minor Damage', 'Moderate Damage', 'Severe Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size = image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print("Your car has - " + train_labels[preds[0]])
    print("Severity assessment complete")
    print("\n")
    
    print("Thank you for using the assessment kit from Chelsi Jain")
    print("More such kits in pipeline...")

Engine - Combining all checks

In [14]:
def engine(img):
    while True:
        try:
            img_224 = prepare_img_224(img)
            g1 = car_categories_check(img_224, first_check)
            
            if g1 is False:
                print("Are you sure that this is a picture of a car? Please upload another picture(try a different angle or lightning).")
                break
            
            g2 = car_damage_check(second_check)
            
            if g2 is False:
                print("Are you sure that your car is damaged? Please submit another picture of damage.")
                print("Hint: Try zooming in/out, using a different angle or different lightning.")
                break
            
            x = location_assessment(third_check)
            y = severity_assessment(forth_check)
            break
            
        except:
            print("Image not accessible. Please try again")
            break

In [15]:
img_path = 'C:/Users/admin/Desktop/Computer Vision + Django/firstCarCheck7.jpg'

In [16]:
engine(img_path)

Validating that this is a picture of car or not
Car Check Passed


Validating that damage exists....
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your car is damaged from Front
Location Check Complete


Validating the severity....
Your car has - Severe Damage
Severity assessment complete


Thank you for using the assessment kit from Chelsi Jain
More such kits in pipeline...
